In [112]:
from keras.layers import Dense,Activation,Dropout,Flatten
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization
from keras.losses import sparse_categorical_crossentropy
from keras.activations import softmax,relu
from keras.optimizers import Adam
from keras.regularizers import l2
import pandas as pd
import numpy as np
from keras.activations import sigmoid

In [113]:
df=pd.read_csv('fer2013.csv')

In [114]:
x_train,x_test,y_train,y_test = [],[],[],[]

for index,row in df.iterrows():
    val = row['pixels'].split()
    try:
        if 'Training' in row['Usage']:
            x_train.append(np.array(val,'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' or 'PrivateTest' in row['Usage']:
            x_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        None

In [115]:
# converting values into array because input should be in array only
x_train = np.array(x_train,'float32')
x_test = np.array(x_test,'float32')
y_train = np.array(y_train,'float32')
y_test = np.array(y_test,'float32')

In [116]:
#normalizing values between 0 and 1
x_train = x_train-np.mean(x_train,axis=0)
x_train = x_train/np.std(x_test,axis=0)

x_test = x_test-np.mean(x_test,axis=0)
x_test = x_test/np.std(x_test,axis=0)

In [117]:
# reshaping data into according to input format
x_train = x_train.reshape(x_train.shape[0],48,48,1)
x_test = x_test.reshape(x_test.shape[0],48,48,1)

In [118]:
len(x_train)

28709

In [119]:
 !pip install -q -U keras-tuner

In [120]:
from kerastuner import RandomSearch
from kerastuner import HyperParameters

In [121]:
def build_model(hp):
    model = Sequential([
        Conv2D(
        filters=hp.Int('filters_1',min_value=64,max_value=128,step=16),
        kernel_size=hp.Choice('kernel_1', values=[3,5]),
        activation='relu',
        input_shape=(48, 48, 1)
        
    ),
    MaxPooling2D(pool_size=(2,2),strides=(2, 2)),
    Dropout(0.4),
    
    #adding second layer to the network
    Conv2D(
        filters=hp.Int('filters_1',min_value=64,max_value=128,step=16),
        kernel_size=hp.Choice('kernel_1', values=[3,5]),
        activation='relu'
        ),
        MaxPooling2D(pool_size=(2,2),strides=(2, 2)),
        Dropout(0.4),
    
    #adding 3rd layer to the network
    Conv2D(
        filters=hp.Int('filters_1',min_value=64,max_value=128,step=16),
        kernel_size=hp.Choice('kernel_1', values=[3,5]),
        activation='relu'
    ),
    MaxPooling2D(pool_size=(2,2),strides=(2, 2)),
    Dropout(0.4),
    
    #adding flatten layer
    Flatten(),
    
    #adding dense layer to the network
    Dense(
        units=hp.Int('units',min_value=2*128,max_value=4*128,step=36),
        activation='relu'
    ),
    Dropout(0.2),
    
    #adding dense layer
    Dense(
        units=hp.Int('units',min_value=2*128,max_value=4*128,step=36),
        activation='relu'
    ),
    Dropout(0.2),
    
    #adding last dense layer
    Dense(7,activation='softmax')])
    
    #compiling the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

In [122]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials = 8,directory='output_build_model')

In [123]:
tuner_search.search(x_train,y_train,epochs=30,validation_split=0.1)

Trial 8 Complete [00h 04m 06s]
val_accuracy: 0.5715778470039368

Best val_accuracy So Far: 0.6105886697769165
Total elapsed time: 00h 33m 09s
INFO:tensorflow:Oracle triggered exit


In [124]:
model= tuner_search.get_best_models(num_models=1)[0]

In [125]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 96)        960       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 96)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 96)        83040     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 96)          8

In [129]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau


early_stopping=EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

call_backs = [
    early_stopping,
    lr_scheduler,
]

In [130]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


img_data_gen = ImageDataGenerator(
     rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True
                
                )
img_data_gen.fit(x_train)

In [131]:
batch_size=32
epochs=100

history = model.fit(x_train,y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                   callbacks=call_backs,
                   validation_data=(x_test,y_test),
                   use_multiprocessing=True
                   )

Epoch 1/100
898/898 [==============================] - 10s 11ms/step - loss: 1.0366 - accuracy: 0.6087 - val_loss: 1.0889 - val_accuracy: 0.5924
Epoch 2/100
898/898 [==============================] - 9s 10ms/step - loss: 1.0224 - accuracy: 0.6148 - val_loss: 1.0852 - val_accuracy: 0.5950
Epoch 3/100
898/898 [==============================] - 9s 10ms/step - loss: 1.0010 - accuracy: 0.6252 - val_loss: 1.0880 - val_accuracy: 0.5940
Epoch 4/100
898/898 [==============================] - 9s 11ms/step - loss: 1.0020 - accuracy: 0.6231 - val_loss: 1.1007 - val_accuracy: 0.5940
Epoch 5/100
898/898 [==============================] - 9s 10ms/step - loss: 0.9922 - accuracy: 0.6275 - val_loss: 1.0955 - val_accuracy: 0.5926
Epoch 6/100
898/898 [==============================] - 9s 10ms/step - loss: 0.9801 - accuracy: 0.6336 - val_loss: 1.0892 - val_accuracy: 0.5952
Epoch 7/100
898/898 [==============================] - 9s 10ms/step - loss: 0.9711 - accuracy: 0.6357 - val_loss: 1.0876 - val_accuracy

In [132]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [133]:
model.save_weights("model.h5")